# Local and Global Analysis of Super-Chandra sample

## Import modules

In [2]:
import hostphot
import sep
from sep import Background
import hostphot.local_photometry as lp
import hostphot.global_photometry as gp
from hostphot.cutouts import download_images
from hostphot.cutouts import get_HST_images
from hostphot.utils import magnitude_calculation
from hostphot.utils import check_filters_validity
import numpy as np
import matplotlib.pyplot as plt
from photutils import CircularAperture,aperture_photometry
from astropy.io import fits
import os
from dotenv import load_dotenv
from astropy.coordinates import SkyCoord
import astropy.wcs as wcs
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
import extinction
from extinction import fitzpatrick99

from photutils import EllipticalAperture,aperture_photometry
from astropy.coordinates import SkyCoord
import astropy.wcs as wcs
import astropy.units as u
from astropy.visualization import simple_norm
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from astropy.visualization import simple_norm
from pylab import rcParams
from matplotlib import rc
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
from hostphot.image_masking import create_mask

#Plotting settings (from Jimenez-Palau et al. 2022)

rc('font', size=16)  
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rcParams['figure.figsize'] = 8, 8
plt.rc('grid', color='gray', linestyle=':')
plt.rc('xtick', direction='in', color='black')
plt.rc('ytick', direction='in', color='black')

#Check the directory and Hostphot version
hostphot.workdir
print('Hostphot version:', hostphot.__version__)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Fontconfig warning: ignoring UTF-8: not a valid region tag


Hostphot version: 2.8.0


/var/folders/3g/b2n38c9s25qbv4f3vy3b_wy80000gn/T/ipykernel_32578/3165301372.py:12: DeprecationWarning: `photutils.CircularAperture` is a deprecated alias for `photutils.aperture.CircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import CircularAperture` to silence this warning.
  from photutils import CircularAperture,aperture_photometry
/var/folders/3g/b2n38c9s25qbv4f3vy3b_wy80000gn/T/ipykernel_32578/3165301372.py:12: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import CircularAperture,aperture_photometry
/var/folders/3g/b2n38c9s25qbv4f3vy3b_wy80000gn/T/ipykernel_32578/3165301372.py:23: DeprecationWarning: `photutils.EllipticalAperture` is a deprecated alias for `photutils.aperture.EllipticalAperture` and will be remov

## Restframe the spectra and correct for galactic extinction (when needed)

Depending on the survey, the cubes are already corrected for MW extinction or not. In case that we have the whole cube, this can be done before performing the aperture integration.

In [3]:
z_list_MUSE=[0.018477,0.0231,0.04321,0.050705,0.066033,0.0742,0.0772]
name_list_MUSE=['ASASSN-15hy','SN2015M' ,'SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs']
av_MUSE=[0.418,0.029,0.110,0.196,0.147,0.226,0.066]
r_v=3.1 

#Depending on the survey, we will need to correct to galactic extinction or not (PMAS is already corrected)
#We start by the MUSE ones

dir_name='/Users/cristinajimenezpalau/Desktop/SCh_sp/'

analysis_type=['global','1kpc','2kpc','see']

for name,z,a_v in zip(name_list_MUSE,z_list_MUSE,av_MUSE):
    for analysis in analysis_type:
    
        obs_wave=[]
        flux=[]
        e_flux=[]

        file = open(dir_name+name+'_obs_'+analysis+'.txt', 'r')
        lines = file.readlines()
        for line in lines:
            obs_wave.append(float(line.split()[0]))
            flux.append(float(line.split()[1]))
            e_flux.append(float(line.split()[2]))

        obs_wave=np.array(obs_wave)
        rest_wave=obs_wave/(1+z)
        number_of_wavelengths = np.floor(max(rest_wave)) - np.ceil(min(rest_wave))
        resampled_wavelength = np.arange(number_of_wavelengths) + np.ceil(rest_wave[0])

        a_lambda = extinction.fitzpatrick99(obs_wave, a_v, r_v)
        correction_factor = 10**(0.4*a_lambda)

        flux=np.array(flux)*correction_factor
        e_flux=np.array(e_flux)*correction_factor

        resampled_flux= np.interp(resampled_wavelength,rest_wave,flux)
        resampled_eflux = np.interp(resampled_wavelength,rest_wave,e_flux)


        data = np.column_stack([resampled_wavelength, resampled_flux,resampled_eflux])
        np.savetxt(dir_name+name+'_rest_'+analysis+'.txt' , data)
    
#Then the PISCO (PMAS) ones. 

z_list_PISCO=[0.2440,0.0214,0.0797,0.0177,0.2440,0.0362,0.0058]
name_list_PISCO=['SN2006gz','SN2009dc','CSS140501','SN2016gxp','SN2003fg','SN2020esm','SN2020hvf']
a_v=[] #Add here the known parameters for extinction
r_v=3.1 #This is fixed
 

dir_name='/Users/cristinajimenezpalau/Desktop/SCh_sp/'
analysis_type=['global','1kpc','2kpc','see']


for name,z in zip(name_list_PISCO,z_list_PISCO):
    for analysis in analysis_type:
    
        obs_wave=[]
        flux=[]
        e_flux=[]

        file = open(dir_name+name+'_obs_'+analysis+'.txt', 'r')
        lines = file.readlines()
        for line in lines:
            obs_wave.append(float(line.split()[0]))
            flux.append(float(line.split()[1]))
            e_flux.append(float(line.split()[2]))

        obs_wave=np.array(obs_wave)
        rest_wave=obs_wave/(1+z)
        number_of_wavelengths = np.floor(max(rest_wave)) - np.ceil(min(rest_wave))
        resampled_wavelength = np.arange(number_of_wavelengths) + np.ceil(rest_wave[0])

        #a_lambda = extinction.fitzpatrick99(obs_wave, a_v, r_v)
        correction_factor = 1

        flux=np.array(flux)*correction_factor
        e_flux=np.array(e_flux)*correction_factor

        resampled_flux= np.interp(resampled_wavelength,rest_wave,flux)
        resampled_eflux = np.interp(resampled_wavelength,rest_wave,e_flux)


        data = np.column_stack([resampled_wavelength, resampled_flux,resampled_eflux])
        np.savetxt(dir_name+name+'_rest_'+analysis+'.txt' , data)


## We plot the restframed spectra/extinction corrected spectra

In [5]:
name_list=['ASASSN-15hy','SN2015M' ,'SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','SN2006gz','SN2009dc','CSS140501','SN2016gxp','SN2003fg','SN2020esm','SN2020hvf']

dir_name='/Users/cristinajimenezpalau/Desktop/SCh_sp/'
for name in name_list:
    
    rcParams['figure.figsize'] = 10, 8
    
    wave_1kpc=[]
    flux_1kpc=[]
    
    wave_2kpc=[]
    flux_2kpc=[]
    
    wave_see=[]
    flux_see=[]
        
    wave_global=[]
    flux_global=[]
    
    
    file1 = open(dir_name+name+'_rest_1kpc.txt', 'r')
    lines1 = file1.readlines()
    for line1 in lines1:
        wave_1kpc.append(float(line1.split()[0]))
        flux_1kpc.append(float(line1.split()[1]))
        
    wave_1kpc=np.array(wave_1kpc)
    flux_1kpc=np.array(flux_1kpc)
    
    
    file3 = open(dir_name+name+'_rest_2kpc.txt', 'r')
    lines3 = file3.readlines()
    for line3 in lines3:
        wave_2kpc.append(float(line3.split()[0]))
        flux_2kpc.append(float(line3.split()[1]))
        
    wave_2kpc=np.array(wave_2kpc)
    flux_2kpc=np.array(flux_2kpc)
    
     
    file4 = open(dir_name+name+'_rest_see.txt', 'r')
    lines4 = file4.readlines()
    for line4 in lines4:
        wave_see.append(float(line4.split()[0]))
        flux_see.append(float(line4.split()[1]))
        
    wave_see=np.array(wave_see)
    flux_see=np.array(flux_see)
        
    
    file2 = open(dir_name+name+'_rest_global.txt', 'r')
    lines2 = file2.readlines()
    for line2 in lines2:
        wave_global.append(float(line2.split()[0]))
        flux_global.append(float(line2.split()[1]))
        
    wave_global=np.array(wave_global)
    flux_global=np.array(flux_global)
    
    ax1=plt.subplot(221)
    ax1.plot(wave_1kpc,flux_1kpc,lw=0.75,color='blue',label='Flux d = 1 kpc')  
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
    ax1.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$]')
    plt.legend(frameon=False)
    
    ax3=plt.subplot(222)
    ax3.plot(wave_2kpc,flux_2kpc,lw=0.75,color='blue',label='Flux d = 2 kpc')  
    ax3.xaxis.set_ticks_position('both')
    ax3.yaxis.set_ticks_position('both')
    plt.legend(frameon=False)
    
    ax2=plt.subplot(223)
    ax2.plot(wave_global,flux_global,lw=0.75,color='blue',label='Global Flux')
    ax2.xaxis.set_ticks_position('both')
    ax2.yaxis.set_ticks_position('both')
    ax2.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ \AA$^{-1}$]')
    ax2.set_xlabel(r'Wavelength [\AA]')
    plt.legend(frameon=False)
    
    ax4=plt.subplot(224)
    ax4.plot(wave_global,flux_global,lw=0.75,color='blue',label=' d = Seeing Flux')
    ax4.xaxis.set_ticks_position('both')
    ax4.yaxis.set_ticks_position('both')
    ax4.set_xlabel(r'Wavelength [\AA]')
    plt.legend(frameon=False)
    plt.savefig(dir_name+name+'_rest_spec.pdf', format='pdf')
    plt.close()

## We download the photometric data 

We download the photometry from the available surveys per each target. When two surveys are available in the same bands, we try to keep the best resolution possible.

Note 1: ASASSN-15hy has Hubble photometry available, but the photometry from Hubble is downloaded manually since HOSTPHOT struggles to do it (Due to the size of the files).

Note 2: I am still pending on getting VISTA photometry, there's a bug pending to be fixed in HOSTPHOT.

In [13]:
z     =[0.003        ,0.005811   ,0.010177  ,0.014903     ,0.01767    ,0.018477     ,0.021391  ,0.0231   ,0.023630,0.031,0.036188   ,0.043212  ,0.050705  ,0.066033  ,0.0742    ,0.0772    ,0.0798     ,0.244]
host_ra =[186.703526264,170.354126 ,305.912333,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134  ,227.684417 ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec=[8.44870123318,3.013929   ,-28.277833,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-11.599439 ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]
name_list   =['SN2022pul'  ,'SN2020hvf','SN2012dn','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2022ilv','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']
survey_list=['GALEX','SDSS','PS1','2MASS','unWISE','UKIDSS','SkyMapper']
#survey_list=['VISTA']

for ra,dec,name in zip(host_ra,host_dec,name_list):
    for survey in survey_list:
        download_images(name, ra, dec, survey=survey)

## We obtain the mask parameters to clean the images

In general we will use PS1 coadded image to obtain the masking. In few cases this may not be feasible the coadded image may be used.

In [14]:
from astropy import units as u
from astropy.coordinates import Angle
from hostphot.coadd import coadd_images
from hostphot.image_masking import create_mask

host_ra_list=[186.703526264,170.354126 ,305.912333,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134  ,227.684417 ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec_list=[8.44870123318,3.013929   ,-28.277833,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-11.599439 ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]

name_list=['SN2022pul'  ,'SN2020hvf','SN2012dn','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2022ilv','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']

survey_list = ['PS1']
coadd_filters_list = ['grizy']

eps = 0.001
mask_params=[]
for survey,coadd_filters in zip(survey_list,coadd_filters_list):
    for name,host_ra,host_dec in zip(name_list,host_ra_list,host_dec_list):
            try:
                coadd_images(name, filters=coadd_filters, survey=survey)

                coadd_mask_params = create_mask(name, host_ra, host_dec,
                                            filt=coadd_filters,survey=survey,
                                            extract_params=True,threshold=3,crossmatch=True,save_plots=True)
                mask_params.append(coadd_mask_params)
           
                
            except FileNotFoundError:
                print('No file for this filter')
            except TypeError:
                print('The galaxy was not identified')

No file for this filter


## We apply the mask to the images

In [15]:
import hostphot.global_photometry as gp
from astropy import units as u
from astropy.coordinates import Angle

from hostphot.utils import plot_fits, get_survey_filters

host_ra_list=[186.703526264,170.354126 ,305.912333,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134  ,227.684417 ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec_list=[8.44870123318,3.013929   ,-28.277833,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-11.599439 ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]

name_list=['SN2022pul'  ,'SN2020hvf','SN2012dn','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2022ilv','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']

survey_list=['PS1','PS1','PS1','PS1','PS1','GALEX','GALEX','SDSS','SkyMapper','2MASS','2MASS','2MASS','UKIDSS','UKIDSS','UKIDSS','UKIDSS','UKIDSS']
filters_list=['g','r','i','z','y','FUV','NUV','u','u','J','H','Ks','Z','Y','J','H','K']
 
for survey,filters in zip(survey_list,filters_list):
            for name,host_ra,host_dec,mask in zip(name_list,host_ra_list,host_dec_list,mask_params):
                try:
                    create_mask(name, host_ra, host_dec,filt=filters,survey=survey,common_params=mask,threshold=3,save_plots=True)
                except FileNotFoundError:
                    print('No file for this filter')
                except ValueError:
                    print('No valid mask')
                except TypeError:
                    print('No valid mask')

No file for this filter
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No valid mask
No file for this filter
No valid mask
No valid mask
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No valid mask
No file for this filter
No valid mask
No file for this filter
No file for this filter
No file for this filter
No file for this filter
No valid mask
No valid mask
No file for this filter
No file for this filter
No valid mask
No file for this filter
No valid mask
No valid mask
No valid mask
No valid mask
No valid mask
No valid mask
No valid mask
No valid mask
No valid mask
No f

## Obtaining local photometry using HOSTPHOT

The local apperture is integrated in a circle of d = 1 and 2 kpc centered in the SN coordinates, correcting MW extinction and using the masks if necesary.

IMPORTANT! We must match the appertures. This means that if we use masks in the images we then need to apply a kernel in the surface projected in the spectrum before integrating the flux in the cubes. For the test done here is not relevant but this needs to be considered.


In [52]:
snra_list  =[170.360208,47.201850,3.644375,302.509792,237.8005,195.13458,272.60971,211.826083,176.18642,48.242667,334.192125,17.71404,180.77875,256.05704,214.07825]
sndec_list =[3.014681,-35.230800,48.252250,-0.739253,25.70778,27.97808,30.99567,-5.127131,-20.52808,-11.711147,15.353925,15.46108,-1.02553333,17.81094,52.24872]

host_ra_list=[170.354126 ,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134   ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec_list=[3.013929  ,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]

z_list=[0.005811 ,0.014903,0.01767,0.018477     ,0.021391  ,0.0231   ,0.023630  ,0.0266     ,0.031      ,0.036188   ,0.043212  ,0.050705  ,0.066033  ,0.0742    ,0.0772    ,0.0798     ,0.244]
name_list=['SN2020hvf','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']

survey_list=['GALEX','SDSS','PS1','2MASS','UKIDSS','SkyMapper']
radii=0.5
for ra,dec,name,z in zip(snra_list,sndec_list,name_list,z_list): 
        for survey in survey_list:            
                results = lp.multi_band_phot(name, ra, dec, z,
                                survey=survey, ap_radii=radii,
                                use_mask=True, correct_extinction=True,
                                save_plots=True,raise_exception=False)

/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources, which might be solved in a future data release.
  warnings.warn(("SkyMapper photometry is not completely trustworthy due to imprecision in "
/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources, which might be solved in a future data release.
  warnings.warn(("SkyMapper photometry is not completely trustworthy due to imprecision in "
/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources

In [25]:
snra_list  =[170.360208,47.201850,3.644375,302.509792,237.8005,195.13458,272.60971,211.826083,176.18642,48.242667,334.192125,17.71404,180.77875,256.05704,214.07825]
sndec_list =[3.014681,-35.230800,48.252250,-0.739253,25.70778,27.97808,30.99567,-5.127131,-20.52808,-11.711147,15.353925,15.46108,-1.02553333,17.81094,52.24872]

host_ra_list=[170.354126 ,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134   ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec_list=[3.013929  ,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]

z_list=[0.005811 ,0.014903,0.01767,0.018477     ,0.021391  ,0.0231   ,0.023630  ,0.0266     ,0.031      ,0.036188   ,0.043212  ,0.050705  ,0.066033  ,0.0742    ,0.0772    ,0.0798     ,0.244]
name_list=['SN2020hvf','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']

survey_list=['GALEX','SDSS','PS1','2MASS','UKIDSS','SkyMapper']
radii=1
for ra,dec,name,z in zip(snra_list,sndec_list,name_list,z_list): 
        for survey in survey_list:            
                results = lp.multi_band_phot(name, ra, dec, z,
                                survey=survey, ap_radii=radii,
                                use_mask=True, correct_extinction=True,
                                save_plots=True,raise_exception=False)

/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources, which might be solved in a future data release.
  warnings.warn(("SkyMapper photometry is not completely trustworthy due to imprecision in "
/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources, which might be solved in a future data release.
  warnings.warn(("SkyMapper photometry is not completely trustworthy due to imprecision in "
/Users/cristinajimenezpalau/anaconda3/envs/hostphot_dev/lib/python3.9/site-packages/hostphot/local_photometry.py:242: UserWarning: SkyMapper photometry is not completely trustworthy due to imprecision in the zeropoint for extended sources

## Obtaining global photometry given a certain apperture

Instead of obtaining the apperture parameters using HOSTPHOT's global photometry, we rely on the apperture parameters given by the appertures which were defined in advance.

Therefore, in this case, we should perform the following tasks:

1. Gets the .fits files available per each target. Depending of how the apertures are defined we may (or may not) use masked images. Remember: the integrated surface should always match.

2. Match the resolution of different surveys to match the appertures. We will make use of pixel scale, since we have the effective radii in arcsec.

3. Obtain the coordinates of the center of the host by using WCS. 

4. Perform the background substraction for the images from certain surveys that require it.

5. Correct MW extinction

6. Integrate the surface apperture using EllipticalApperture. In some cases, the images are rotated. Instead of rotating the images, what we do is to rotate the paralactic angle when defining the elipse.

7. We obtain the AB/Vega magnitudes and uncertainties from the output fluxes from EllipticalApperture using HOSTPHOT's magnitude_calculation.

8. Once we have the AB/Vega magnitudes we proceed as we did for local photometry, converting Vega magnitudes to AB and defining a quality cut. 

9. Generating the .pho file


In [6]:
import os
import pandas as pd
import pathlib
from hostphot.dust import calc_extinction
from hostphot.utils import survey_pixel_scale
from hostphot.utils import get_image_exptime
from hostphot.utils import calc_sky_unc
from hostphot.utils import magnitude_calculation
import sep
from hostphot.utils import get_image_gain
from sep import Background

#We will try to read all the files with mask


analysis='global'

path_global = '/Users/cristinajimenezpalau/Desktop/Super-Chandra/'
name_list=['SN2020hvf','ASASSN-15pz','SN2016gxp','ASASSN-15hy','SN2009dc','SN2015M','SN2006gz','SN2020esm','SN2013ao','LSQ12gpw','LSQ14fmg','SN2007if','CSP13abs','CSS140501','SN2003fg']
host_ra_list=[170.354126 ,47.206370    ,3.645482   ,302.5126     ,237.80523 ,195.13535,272.6134   ,211.823042 ,176.18625 ,48.24505  ,334.1922  ,17.7142   ,180.77828 ,256.05478  ,214.07877 ]
host_dec_list=[3.013929  ,-35.235921   ,48.255687  ,-0.7419      ,25.70232  ,27.97611 ,31.0037   ,-5.123861  ,-20.52840 ,-11.71085 ,15.353735 ,15.46098  ,-1.025665 ,17.81604   ,52.248592 ]
z_list=[0.005811 ,0.014903,0.01767,0.018477     ,0.021391  ,0.0231   ,0.023630  ,0.0266     ,0.031      ,0.036188   ,0.043212  ,0.050705  ,0.066033  ,0.0742    ,0.0772    ,0.0798     ,0.244]
reff_list=[22.5, 6.25,19.0,14.4,19.75,7.5 ,30.9 ,np.nan     ,np.nan     ,16.25      ,3.14      ,np.nan     ,13.78     ,5.78      ,1.25      ,2.5       ,13.26      ,6.25 ]
barat_list=[0.55,0.75, 0.66,0.9233476598 ,0.52185276,0.72     ,0.75282428,np.nan     ,np.nan     ,0.90       ,0.78907142,np.nan     ,0.8564391 ,0.86685623,0.65      ,0.7       ,0.502504768,0.75]
pa_list=[10.0,60,130,65.0030263988,107.748628,132.     ,120.724966,np.nan     ,np.nan     ,50.0       ,175.071728,np.nan     ,175.81067 ,32.818411 ,170.      ,170.      ,80.9833227 ,60.]

for name,host_ra,host_dec,reff,barat,pa,z in zip(name_list,host_ra_list,host_dec_list,reff_list,barat_list,pa_list,z_list):
    path = '/Users/cristinajimenezpalau/Desktop/Super-Chandra/images/'+name+'/'
    #We avoid using the masked images since Lluis didn't use them
    files=[f for f in os.listdir(path) if f.endswith(".fits") and f.startswith("masked") and not f.endswith("grizy.fits")]
    print('')
    print('The existing HOSTPHOT masked files for the target' ,name,  'are',str(files))
    files=np.array(files)
    
    #We need to delete the extension FITS to extract the info
    
    files_info = [x[:-5] for x in files]
    print('')
    print(files_info)
    survey_info = []
    filter_info=[]
    for i in files_info:
        a = i.split('_')
        survey = a[1]
        filters=a[2]
        survey_info.append(survey)
        filter_info.append(filters)
    print('')
    print('The surveys of the files are',survey_info)
    print('')
    print('The filters of the files are',filter_info)
    magnitudes=[]
    err_mag=[]
    i_survey=[]
    i_filter=[]
    for i,f in enumerate(files):
        img=fits.open(path+f)
        w1=wcs.WCS(img[0].header)
        s1=SkyCoord(host_ra,host_dec,unit=(u.deg,u.deg))
        xc1,yc1=s1.to_pixel(w1)
        xc1=int(np.round(xc1));yc1=int(np.round(yc1))
        print('The center of the host',name, 'is at pixel',yc1,xc1,'at the ',survey_info[i],'-',filter_info[i],'image')
        pixel_scale=survey_pixel_scale(survey_info[i],filter_info[i])
        exp_time=get_image_exptime(img[0].header, survey_info[i])
        try:
            reff_survey=reff/pixel_scale
            if survey_info[i]=='UKIDSS' or survey_info[i]=='SDSS':
                rad_pa=pa*np.pi/180
                rad_pa=rad_pa+(np.pi/4)
            else:
                rad_pa=pa*np.pi/180
            a=(2.5)*reff_survey
            b=a*barat        
            data=img[0].data
            sigma=calc_sky_unc(data,exp_time)

            data = data.astype(np.float64)
            bkg = sep.Background(data)
            bkg_rms = bkg.globalrms
            if survey_info[i]=='UKIDSS'or survey_info[i]=='2MASS'or survey_info[i]=='SkyMapper':
                data = np.copy(data - bkg)

            A_ext = calc_extinction(filter_info[i], survey_info[i], host_ra, host_dec)
            data *= 10**(0.4*A_ext)

            aperture= EllipticalAperture([xc1,yc1], a=a, b=b,theta=rad_pa)
            phot_table = aperture_photometry(data, aperture,error=sigma)

            norm = simple_norm(data, 'sqrt', percent=99)
            plt.imshow(data, cmap = 'viridis', origin = 'lower',norm=norm,interpolation = 'nearest')
            plt.ylabel(r'y[arcsec]')
            plt.xlabel(r'x[arcsec]')
            aperture.plot(color='white', lw=1,label='Photometry aperture')
            plt.legend()
            plt.savefig(path+name+'_'+'global_'+survey_info[i]+'_'+filter_info[i]+'.pdf')
            plt.close()
    
            flux=float(phot_table['aperture_sum'])
            flux_err=float(phot_table['aperture_sum_err'])

            gain=get_image_gain(img[0].header,survey_info[i])

            mag,mag_err,flux,flux_err,zp=magnitude_calculation(flux, flux_err, survey_info[i], filt=filter_info[i], ap_area=np.pi*a*b, header=img[0].header, bkg_rms=0.0)
            magnitudes.append(mag)
            err_mag.append(mag_err)
            i_survey.append(survey_info[i])
            i_filter.append(filter_info[i])
            
        except ValueError:
                print('No Elipse available for this target', name)

        data = pd.read_csv('mag_vega.csv',sep=',',index_col=False)
        data_vega=data.dropna(axis=1, how='all')

        surveys_vega=data_vega['survey']; surveys_vega=np.array(surveys_vega)
        filters_vega=data_vega['filters']; filters_vega=np.array(filters_vega)
        conv_vega=data_vega['mag_sys_conv']; conv_vega=np.array(conv_vega)

        content_lines=[]    
        for ii in range(len(magnitudes)):
            if not np.isnan(magnitudes[ii]) and not np.isnan(err_mag[ii]) and err_mag[ii]<1 :
                      for kk in range(len(conv_vega)):
                         if surveys_vega[kk]==i_survey[ii] and filters_vega[kk]==i_filter[ii]:
                                magnitudes[ii]=magnitudes[ii]+conv_vega[kk]
                                print('The magnitude from filter',i_filter[ii], 'and survey', i_survey[ii],'from target' ,name, 'has changed from Vega to AB system')   
                      content_lines.append(i_survey[ii]+ ','+i_filter[ii]+','+str(magnitudes[ii])+','+str(err_mag[ii]))
        with open('/Users/cristinajimenezpalau/Desktop/Super-Chandra/'+name+'_'+analysis+'.txt', 'w') as f:
            preface_lines=['survey,filter,magnitudes,errors']
            for preface_line in preface_lines:
                f.write(preface_line)
                f.write('\n')   
            for content_line in content_lines:
                f.write(content_line)
                f.write('\n')
            f.close()                 


The existing HOSTPHOT masked files for the target SN2020hvf are ['masked_UKIDSS_H.fits', 'masked_GALEX_NUV.fits', 'masked_PS1_i.fits', 'masked_GALEX_FUV.fits', 'masked_SDSS_u.fits', 'masked_2MASS_H.fits', 'masked_PS1_z.fits', 'masked_PS1_y.fits', 'masked_UKIDSS_Y.fits', 'masked_2MASS_Ks.fits', 'masked_2MASS_J.fits', 'masked_SkyMapper_u.fits', 'masked_UKIDSS_J.fits', 'masked_UKIDSS_K.fits', 'masked_PS1_g.fits', 'masked_PS1_r.fits']

['masked_UKIDSS_H', 'masked_GALEX_NUV', 'masked_PS1_i', 'masked_GALEX_FUV', 'masked_SDSS_u', 'masked_2MASS_H', 'masked_PS1_z', 'masked_PS1_y', 'masked_UKIDSS_Y', 'masked_2MASS_Ks', 'masked_2MASS_J', 'masked_SkyMapper_u', 'masked_UKIDSS_J', 'masked_UKIDSS_K', 'masked_PS1_g', 'masked_PS1_r']

The surveys of the files are ['UKIDSS', 'GALEX', 'PS1', 'GALEX', 'SDSS', '2MASS', 'PS1', 'PS1', 'UKIDSS', '2MASS', '2MASS', 'SkyMapper', 'UKIDSS', 'UKIDSS', 'PS1', 'PS1']

The filters of the files are ['H', 'NUV', 'i', 'FUV', 'u', 'H', 'z', 'y', 'Y', 'Ks', 'J', 'u', 'J',

The center of the host SN2016gxp is at pixel 360 359 at the  PS1 - i image
The center of the host SN2016gxp is at pixel 59 59 at the  GALEX - FUV image
The center of the host SN2016gxp is at pixel 90 90 at the  2MASS - H image
The magnitude from filter H and survey 2MASS from target SN2016gxp has changed from Vega to AB system
The center of the host SN2016gxp is at pixel 360 359 at the  PS1 - z image
The magnitude from filter H and survey 2MASS from target SN2016gxp has changed from Vega to AB system
The center of the host SN2016gxp is at pixel 360 359 at the  PS1 - y image
The magnitude from filter H and survey 2MASS from target SN2016gxp has changed from Vega to AB system
The center of the host SN2016gxp is at pixel 90 90 at the  2MASS - Ks image
The magnitude from filter H and survey 2MASS from target SN2016gxp has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target SN2016gxp has changed from Vega to AB system
The center of the host SN2016gxp is 

The magnitude from filter H and survey UKIDSS from target SN2009dc has changed from Vega to AB system
The magnitude from filter H and survey 2MASS from target SN2009dc has changed from Vega to AB system
The magnitude from filter Y and survey UKIDSS from target SN2009dc has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target SN2009dc has changed from Vega to AB system
The center of the host SN2009dc is at pixel 68 90 at the  2MASS - J image
The magnitude from filter H and survey UKIDSS from target SN2009dc has changed from Vega to AB system
The magnitude from filter H and survey 2MASS from target SN2009dc has changed from Vega to AB system
The magnitude from filter Y and survey UKIDSS from target SN2009dc has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target SN2009dc has changed from Vega to AB system
The magnitude from filter J and survey 2MASS from target SN2009dc has changed from Vega to AB system
The center 

The center of the host SN2006gz is at pixel 60 59 at the  GALEX - FUV image
The center of the host SN2006gz is at pixel 90 90 at the  2MASS - H image
The magnitude from filter H and survey 2MASS from target SN2006gz has changed from Vega to AB system
The center of the host SN2006gz is at pixel 360 360 at the  PS1 - z image
The magnitude from filter H and survey 2MASS from target SN2006gz has changed from Vega to AB system
The center of the host SN2006gz is at pixel 360 360 at the  PS1 - y image
The magnitude from filter H and survey 2MASS from target SN2006gz has changed from Vega to AB system
The center of the host SN2006gz is at pixel 90 90 at the  2MASS - Ks image
The magnitude from filter H and survey 2MASS from target SN2006gz has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target SN2006gz has changed from Vega to AB system
The center of the host SN2006gz is at pixel 90 90 at the  2MASS - J image
The magnitude from filter H and survey 2MASS fr

The magnitude from filter H and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter J and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The center of the host LSQ12gpw is at pixel 360 360 at the  PS1 - g image
The magnitude from filter H and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter J and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The center of the host LSQ12gpw is at pixel 360 360 at the  PS1 - r image
The magnitude from filter H and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target LSQ12gpw has changed from Vega to AB system
The magnitude from filter J and survey 2M

The magnitude from filter Y and survey UKIDSS from target CSP13abs has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target CSP13abs has changed from Vega to AB system
The magnitude from filter J and survey 2MASS from target CSP13abs has changed from Vega to AB system
The center of the host CSP13abs is at pixel 182 181 at the  SkyMapper - u image
The magnitude from filter Y and survey UKIDSS from target CSP13abs has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target CSP13abs has changed from Vega to AB system
The magnitude from filter J and survey 2MASS from target CSP13abs has changed from Vega to AB system
The center of the host CSP13abs is at pixel 448 447 at the  UKIDSS - J image
The magnitude from filter Y and survey UKIDSS from target CSP13abs has changed from Vega to AB system
The magnitude from filter Ks and survey 2MASS from target CSP13abs has changed from Vega to AB system
The magnitude from filter J a